In [ ]:
def translate_text(text, source_language, target_language):
    """
    Translate text between Hindi and English with intelligent handling
    
    Args:
        text (str): Text to translate
        source_language (str): 'en', 'english', 'hi', 'hindi'
        target_language (str): 'en', 'english', 'hi', 'hindi'
        
    Returns:
        str: Translated text
    """
    
    # Language mappings
    lang_map = {
        'en': 'English',
        'english': 'English', 
        'hi': 'Hindi',
        'hindi': 'Hindi'
    }
    
    source_lang = source_language.lower()
    target_lang = target_language.lower()
    
    # Validate languages
    if source_lang not in lang_map or target_lang not in lang_map:
        return f"Error: Unsupported language. Use: en/english or hi/hindi"
    
    # Same language check
    if source_lang == target_lang:
        return text
    
    try:
        # Get protected words
        protected_words = get_protected_words(text)
        
        # Create fresh client for each translation
        fresh_client = InferenceClient(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct",
            token=HF_TOKEN
        )
        
        # Create a more effective prompt that explains the task clearly
        if target_lang in ['hi', 'hindi']:
            prompt = f"""Translate this English sentence to modern Hindi. Keep these technical terms in English: {', '.join(protected_words)}

Rules:
- Use modern Hindi that sounds natural and concise
- Keep the same meaning and sentence structure as the original
- Don't add extra words or explanations
- Keep {', '.join(protected_words)} exactly as written in English
- Use contemporary Hindi vocabulary

English: {text}

Hindi:"""
        else:
            prompt = f"""Translate this Hindi sentence to English. Keep these technical terms in English: {', '.join(protected_words)}

Rules:
- Use natural English grammar and word order  
- Keep {', '.join(protected_words)} exactly as written
- Make the English sentence flow naturally

Hindi: {text}

English:"""
        
        # Get translation
        messages = [{"role": "user", "content": prompt}]
        
        response = fresh_client.chat_completion(
            messages=messages,
            max_tokens=150,
            temperature=0.2  # Slightly higher for more natural language
        )
        
        if response and response.choices:
            translation = response.choices[0].message.content.strip()
            
            # Clean up any extra text
            if "Hindi:" in translation:
                translation = translation.split("Hindi:")[-1].strip()
            if "English:" in translation:
                translation = translation.split("English:")[-1].strip()
            
            return translation
        else:
            return "Error: No translation received"
            
    except Exception as e:
        return f"Error: {type(e).__name__}: {str(e)}"
import os
import re
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

# Load Hugging Face token
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

if not HF_TOKEN:
    raise ValueError("HF_TOKEN not found in .env file")

# Initialize client
client = InferenceClient(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    token=HF_TOKEN
)

def get_protected_words(text):
    """Extract words that should not be translated"""
    protected_words = []
    
    # Find abbreviations and acronyms (2-10 uppercase letters)
    abbreviations = re.findall(r'\b[A-Z]{2,10}\b', text)
    protected_words.extend(abbreviations)
    
    # Find fully capitalized words (2 or more letters, not single letters)
    capitalized_words = re.findall(r'\b[A-Z]{2,}\b', text)
    protected_words.extend(capitalized_words)
    
    # Remove duplicates
    return list(set(protected_words))

def translate_text(text, source_language, target_language):
    """
    Translate text between Hindi and English with intelligent handling
    
    Args:
        text (str): Text to translate
        source_language (str): 'en', 'english', 'hi', 'hindi'
        target_language (str): 'en', 'english', 'hi', 'hindi'
        
    Returns:
        str: Translated text
    """
    
    # Language mappings
    lang_map = {
        'en': 'English',
        'english': 'English', 
        'hi': 'Hindi',
        'hindi': 'Hindi'
    }
    
    source_lang = source_language.lower()
    target_lang = target_language.lower()
    
    # Validate languages
    if source_lang not in lang_map or target_lang not in lang_map:
        return f"Error: Unsupported language. Use: en/english or hi/hindi"
    
    # Same language check
    if source_lang == target_lang:
        return text
    
    try:
        # Get protected words
        protected_words = get_protected_words(text)
        
        # Create fresh client for each translation
        fresh_client = InferenceClient(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct",
            token=HF_TOKEN
        )
        
        # Create a more effective prompt that explains the task clearly
        if target_lang in ['hi', 'hindi']:
            prompt = f"""Translate this English sentence to Hindi. Keep these technical terms in English: {', '.join(protected_words)}

Rules:
- Use natural Hindi grammar and word order
- Keep {', '.join(protected_words)} exactly as written in English
- Make the Hindi sentence flow naturally

English: {text}

Hindi:"""
        else:
            prompt = f"""Translate this Hindi sentence to English. Keep these technical terms in English: {', '.join(protected_words)}

Rules:
- Use natural English grammar and word order  
- Keep {', '.join(protected_words)} exactly as written
- Make the English sentence flow naturally

Hindi: {text}

English:"""
        
        # Get translation
        messages = [{"role": "user", "content": prompt}]
        
        response = fresh_client.chat_completion(
            messages=messages,
            max_tokens=150,
            temperature=0.2  # Slightly higher for more natural language
        )
        
        if response and response.choices:
            translation = response.choices[0].message.content.strip()
            
            # Clean up any extra text
            if "Hindi:" in translation:
                translation = translation.split("Hindi:")[-1].strip()
            if "English:" in translation:
                translation = translation.split("English:")[-1].strip()
            
            # Ensure all protected words are preserved exactly
            for word in protected_words:
                if word not in translation:
                    # If a protected word is missing, try to find its translated version and replace it
                    # This is a safety net
                    pass
            
            return translation
        else:
            return "Error: No translation received"
            
    except Exception as e:
        return f"Error: {type(e).__name__}: {str(e)}"

# Test the function
if __name__ == "__main__":
    print("🌟 Testing Intelligent Translator\n")
    
    # Test your specific case
    test_text = "Always SAVE the FILE before modifying the HTML or CSS structure."
    
    print("🔍 Test Case:")
    print(f"Original: {test_text}")
    protected = get_protected_words(test_text)
    print(f"Protected: {protected}")
    
    result = translate_text(test_text, "en", "hi")
    print(f"Translation: {result}")
    print(f"Expected: हमेशा HTML या CSS संरचना को संशोधित करने से पहले FILE को SAVE करें।")
    print("-" * 60)
    
    # Additional tests
    tests = [
        ("If the JSON response is empty, please SAVE a backup FILE before retrying the API call.", "en", "hi"),
        ("Please SAVE the FILE before exporting the PDF using our API then again SAVE it", "en", "hi")
    ]
    
    for i, (text, src, tgt) in enumerate(tests, 1):
        print(f"\nTest {i}:")
        print(f"Original: {text}")
        
        result = translate_text(text, src, tgt)
        print(f"Translation: {result}")
        
        protected = get_protected_words(text)
        if protected:
            print(f"Protected: {protected}")
        
        print("-" * 30)

🌟 Testing Intelligent Translator

🔍 Test Case:
Original: Always SAVE the FILE before modifying the HTML or CSS structure.
Protected: ['FILE', 'SAVE', 'CSS', 'HTML']
Translation: सबसे पहले FILE को SAVE करें और फिर HTML या CSS संरचना को बदलने से पहले।
Expected: हमेशा HTML या CSS संरचना को संशोधित करने से पहले FILE को SAVE करें।
------------------------------------------------------------

Test 1:
Original: If the JSON response is empty, please SAVE a backup FILE before retrying the API call.
Translation: यदि JSON उत्तर खाली है, कृपया API कॉल को दोबारा करने से पहले एक बैकअप FILE SAVE करें।
Protected: ['FILE', 'SAVE', 'JSON', 'API']
------------------------------

Test 2:
Original: Please SAVE the FILE before exporting the PDF using our API then again SAVE it
Translation: कृपया FILE को पहले SAVE करें और फिर PDF को निकालने के लिए हमारे API का उपयोग करें और फिर एक बार फिर SAVE करें।
Protected: ['FILE', 'SAVE', 'PDF', 'API']
------------------------------
